# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import numpy as np
import os
import random
import re
import torch
import torch.nn.functional as F

from torch import nn, optim
from tqdm import tqdm

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""
print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    # Source: https://urlregex.com/
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\S+', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@\S+', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(token) for token in tokens if len(token) > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
tokenized = [preprocess(message) for message in tqdm(messages)]

100%|██████████| 1548010/1548010 [02:38<00:00, 9758.81it/s] 


Check some tokenized messages

In [7]:
for i in range(6):
    print(f'Message {i}:')
    print(f'\tRaw message: {messages[i]}')
    print(f'\tTokenized message: {tokenized[i]}')

Message 0:
	Raw message: $FITB great buy at 26.00...ill wait
	Tokenized message: ['great', 'buy', 'at', 'ill', 'wait']
Message 1:
	Raw message: @StockTwits $MSFT
	Tokenized message: []
Message 2:
	Raw message: #STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating
	Tokenized message: ['staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy']
Message 3:
	Raw message: $AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.
	Tokenized message: ['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits']
Message 4:
	Raw message: $AMD reveal yourself!
	Tokenized message: ['reveal', 'yourself']
Message 5:
	Raw message: $AAPL Why the drop? I warren Buffet taking out his 

Remove empty tokens such as message 1 and realing the labels

In [8]:
print(f"Before removing empty tokens: {len(tokenized)}")

Before removing empty tokens: 1548010


In [9]:
non_empty_tokens = [idx for idx, token in enumerate(tokenized) if len(token) > 0]
tokenized = [tokenized[idx] for idx in non_empty_tokens]
sentiments = [sentiments[idx] for idx in non_empty_tokens]

In [10]:
print(f"After removing empty tokens: {len(tokenized)}")

After removing empty tokens: 1499394


In [11]:
for i in range(6):
    print(f'Message {i}:')
    print(f'\tRaw message: {messages[i]}')
    print(f'\tTokenized message: {tokenized[i]}')

Message 0:
	Raw message: $FITB great buy at 26.00...ill wait
	Tokenized message: ['great', 'buy', 'at', 'ill', 'wait']
Message 1:
	Raw message: @StockTwits $MSFT
	Tokenized message: ['staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy']
Message 2:
	Raw message: #STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating
	Tokenized message: ['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits']
Message 3:
	Raw message: $AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.
	Tokenized message: ['reveal', 'yourself']
Message 4:
	Raw message: $AMD reveal yourself!
	Tokenized message: ['why', 'the', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position']


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [12]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

tokenized_words = [word for twit in tokenized for word in twit]
bow = Counter(tokenized_words)

Print most common words

In [13]:
print(bow.most_common(5))

[('the', 398762), ('to', 379502), ('is', 284868), ('for', 273542), ('on', 241673)]


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [14]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
num_tokenized_words = len(tokenized_words)
freqs = {key: value / num_tokenized_words for key, value in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000005

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word[0] for word in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words)

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'are', 'you', 'be', 'that', 'short', 'what', 'today']


6684

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [15]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""
# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: ii + 1 for ii, word in enumerate(filtered_words)}

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: word for ii, word in enumerate(filtered_words)}

# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in message if word in vocab] for message in tqdm(tokenized)]

100%|██████████| 1499394/1499394 [00:08<00:00, 176078.90it/s]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [16]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [17]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19891943133474657

Finally let's convert our tokens into integer ids which we can pass to the network.

In [18]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [19]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [20]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # Setup embedding layer
        self.embedding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embed_size)
        
        # Setup LSTM layers
        self.lstm = nn.LSTM(
            input_size=self.embed_size, 
            hidden_size=self.lstm_size, 
            num_layers=self.lstm_layers, 
            dropout=self.dropout,
            batch_first=False
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # Linear and softmax layers
        self.fc = nn.Linear(in_features=self.lstm_size, out_features=self.output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden_state = (
                weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda()
            )
        else:
            hidden_state = (
                weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_()
            )
            
        return hidden_state


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        # Embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
    
        # Stack up lstm outputs
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        lstm_out = lstm_out[-1]
        
        # Dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # Softmax functiontrain
        logps = self.log_softmax(out)
        
        # Reshape to be batch_size first
        #log_softmax_out = log_softmax_out.view(batch_size, -1)
        #log_softmax_out = log_softmax_out[:, -1] # get last batch of labels
        
        # Return last softmax output and hidden state
        return logps, hidden_state

### View Model

In [21]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)

if train_on_gpu:
    input = input.cuda()
    model.cuda()

hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.9500, -1.5144, -1.9158, -1.5678, -1.2657],
        [-1.9845, -1.4925, -1.9074, -1.5668, -1.2710],
        [-1.9189, -1.4674, -1.9958, -1.6132, -1.2463],
        [-1.9247, -1.4902, -1.9533, -1.6400, -1.2277]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [22]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [23]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

split = int(0.8 * len(token_ids))

train_features = token_ids[:split]
valid_features = token_ids[split:]
train_labels = sentiments[:split]
valid_labels = sentiments[split:]

In [24]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=40, batch_size=512)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(512)
if train_on_gpu:
    text_batch = text_batch.cuda()
    model.cuda()
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(6685, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [29]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 2
batch_size = 512
learning_rate = 0.001
sequence_length = 40

# Gradient clipping
clip = 5 

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# move model to GPU, if available
if(train_on_gpu):
    model.cuda()

model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=sequence_length, shuffle=True):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        if(train_on_gpu):
            text_batch, labels = text_batch.cuda(), labels.cuda()
        
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        hidden = tuple([each.data for each in hidden])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # Zero accumulated gradients
        model.zero_grad()

        # Get the output from the model
        output, hidden = model.forward(text_batch, hidden)

        # Calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        # Loss stats
        if steps % print_every == 0:
            model.eval()
            val_losses = []
            val_accuracy = []
            
            # Get validation loss
            val_hidden = model.init_hidden(labels.shape[0])

            for val_text_batch, val_labels in dataloader(valid_features, valid_labels, batch_size=batch_size, sequence_length=sequence_length):                    
                val_text_batch, val_labels = val_text_batch.to(device), val_labels.to(device)
                
                if(train_on_gpu):
                    val_text_batch, val_labels = val_text_batch.cuda(), val_labels.cuda()
                
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_hidden = tuple([each.data for each in val_hidden])       
                for each in val_hidden:
                    each.to(device)
                
                # Get validation losses
                val_logps, hidden = model.forward(val_text_batch, val_hidden)
                val_loss = criterion(val_logps.squeeze(), val_labels)
                val_losses.append(val_loss.item())
                
                # Get validation accuracy
                val_ps = torch.exp(val_logps)
                top_p, top_class = val_ps.topk(1, dim=1)
                equals = top_class == val_labels.view(*top_class.shape)
                val_accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())

            model.train()
            print(f"Epoch: {epoch + 1}/{epochs}...",
                  f"Step: {steps}...",
                  f"Loss: {loss.item():.6f}...",
                  f"Val Loss: {np.mean(val_losses):.6f}",
                  f"Val Accuracy: {np.mean(val_accuracy):.4f}")

Starting epoch 1
Epoch: 1/2... Step: 100... Loss: 0.684868... Val Loss: 0.797609 Val Accuracy: 0.6937
Epoch: 1/2... Step: 200... Loss: 0.715787... Val Loss: 0.789611 Val Accuracy: 0.6963
Epoch: 1/2... Step: 300... Loss: 0.704031... Val Loss: 0.784323 Val Accuracy: 0.7002
Epoch: 1/2... Step: 400... Loss: 0.707527... Val Loss: 0.789579 Val Accuracy: 0.6986
Epoch: 1/2... Step: 500... Loss: 0.731196... Val Loss: 0.785261 Val Accuracy: 0.6965
Epoch: 1/2... Step: 600... Loss: 0.730949... Val Loss: 0.790367 Val Accuracy: 0.6984
Epoch: 1/2... Step: 700... Loss: 0.724594... Val Loss: 0.786069 Val Accuracy: 0.6978
Epoch: 1/2... Step: 800... Loss: 0.705503... Val Loss: 0.774035 Val Accuracy: 0.7020
Epoch: 1/2... Step: 900... Loss: 0.683036... Val Loss: 0.792316 Val Accuracy: 0.7007
Epoch: 1/2... Step: 1000... Loss: 0.758084... Val Loss: 0.791313 Val Accuracy: 0.7008
Epoch: 1/2... Step: 1100... Loss: 0.746152... Val Loss: 0.783259 Val Accuracy: 0.6995
Epoch: 1/2... Step: 1200... Loss: 0.757780... 

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [70]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]

    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1)
    
    if train_on_gpu:
        text_input = text_input.cuda()
        model.cuda()

    # Get the NN output
    hidden = model.init_hidden(text_input.size(1))
    logps, _ = model.forward(text_input, hidden)

    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps).cpu().detach().numpy()
    
    return pred

In [71]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.cuda()
prediction = predict(text, model, vocab)

In [72]:
prediction

array([[ 0.00161321,  0.04018419,  0.02140614,  0.68562466,  0.25117174]], dtype=float32)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

The model predicts that the text has a positive sentiment (68.6%). The uncertainty of the prediction is 31.4%

In [73]:
prediction_class = np.argmax(prediction)-2
prediction_accuracy = np.max(prediction)
prediction_uncertainty = 1 - prediction_accuracy

print(f'The model predicts that the tweet has a positive sentiment (class {prediction_class}):')
print(f'\t- Accuracy: {prediction_accuracy:.2%}')
print(f'\t- Uncertainty: {prediction_uncertainty:.2%}')

The model predicts that the tweet has a positive sentiment (class 1):
	- Accuracy: 68.56%
	- Uncertainty: 31.44%


Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [74]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [75]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [76]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [77]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': array([[ 0.08340614,  0.13722901,  0.16451015,  0.25845298,  0.3564018 ]], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.